### Merge all data to use in optimisation 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import os

In [2]:
# define paths
data_path = 'data'
output_path = 'output'

# set parent folder as working directory
os.chdir('..')
os.getcwd()

'c:\\Users\\isaldiviagonzatti\\Downloads\\gitFiles\\MScThesisCode\\FLP'

In [3]:
# load distance matrix, orig w/ additional distance, pal data (qty, geodata)
distM = pd.read_csv(os.path.join(output_path, "distMatrix.csv"), index_col=0)
orig =  pd.read_csv(os.path.join(output_path, "nearestnodePAL.csv"), header=None).T
pal = os.path.join(output_path, "palNorth.gpkg")

In [4]:
# rename and transform orig data
orig = orig.rename(columns={0: 'osmidOrig', 1: 'distance'})
orig.osmidOrig = orig.osmidOrig.astype(np.int64)  

In [5]:
# read gdp file
palPoint = gpd.read_file(pal) 
# define lon, lat points and osmid from orig 
# Note that here we assume (and check) that osmid column follows same order as provided when
# calculating nearest node. There must be a way of getting the geodata for which osmIDs were generated
# but we did not look into it
palPoint['xlon']=palPoint['geometry'].x
palPoint['ylat']=palPoint['geometry'].y
palPoint[['osmidOrig', 'fieldDist']] = orig[['osmidOrig', 'distance']]

### We hadn't divided the PAL over 2 to calculate yearly production
palPoint['leavesTonne'] = palPoint['leavesTonne']/2

In [6]:
# aggregate PAL data based on nodes 
palAgg = pd.DataFrame(palPoint.groupby('osmidOrig', as_index=False).agg(
                                            osmidOrig=('osmidOrig','mean'), 
                                            areaHa= ('areaHa','sum'),
                                            leavesTonne=('leavesTonne','sum'),
                                            distance = ('fieldDist', 'sum')
                                        )
                     )

palAgg.osmidOrig = palAgg.osmidOrig.astype(np.int64)  

In [7]:
# merge distances with PAL data
optiData = distM.merge(palAgg, left_on='origs', right_on='osmidOrig')

In [8]:
# calculate total distance (road + field centroids to node) and cost per trip
optiData['totalDist'] = (optiData.path_length + optiData.distance)/1000

In [9]:
#save data and look at result
optiData.to_csv(os.path.join(output_path, "optiData.csv"))
optiData.head()

,origs,dests,path_length,osmidOrig,areaHa,leavesTonne,distance,totalDist
0,3610149725,2656380182,157664.026,3610149725,15.49063,1270.231645,319.129599,157.983156
1,3610149725,5294901777,83956.035,3610149725,15.49063,1270.231645,319.129599,84.275165
2,3610149725,1321571077,201954.500,3610149725,15.49063,1270.231645,319.129599,202.273630
3,3610149725,3677820110,75493.804,3610149725,15.49063,1270.231645,319.129599,75.812934
4,3610149725,3510580697,21814.095,3610149725,15.49063,1270.231645,319.129599,22.133225
